# Set Membership

The cell below defines two **abstract classes**: the first represents a set and basic insert/search operations on it. You will need to impement this API four times, to implement (1) sequential search, (2) binary search tree, (3) balanced search tree, and (4) bloom filter. The second defines the synthetic data generator you will need to implement as part of your experimental framework. <br><br>**Do NOT modify the next cell** - use the dedicated cells further below for your implementation instead. <br>

In [1]:
# DO NOT MODIFY THIS CELL

from abc import ABC, abstractmethod  

# abstract class to represent a set and its insert/search operations
class AbstractSet(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # inserts "element" in the set
    # returns "True" after successful insertion, "False" if the element is already in the set
    # element : str
    # inserted : bool
    @abstractmethod
    def insertElement(self, element):     
        inserted = False
        return inserted   
    
    # checks whether "element" is in the set
    # returns "True" if it is, "False" otherwise
    # element : str
    # found : bool
    @abstractmethod
    def searchElement(self, element):
        found = False
        return found    
    
    
    
# abstract class to represent a synthetic data generator
class AbstractTestDataGenerator(ABC):
    
    # constructor
    @abstractmethod
    def __init__(self):
        pass           
        
    # creates and returns a list of length "size" of strings
    # size : int
    # data : list<str>
    @abstractmethod
    def generateData(self, size):     
        data = [""]*size
        return data

Use the cell below to define any auxiliary data structure and python function you may need. Leave the implementation of the main API to the next code cells instead.

In [2]:
# ADD AUXILIARY DATA STRUCTURE DEFINITIONS AND HELPER CODE HERE



Use the cell below to implement the requested API by means of **sequential search**.

In [3]:
class SequentialSearchSet(AbstractSet):
    
    def __init__(self):
        # ADD YOUR CODE HERE
        self.data = []
        self.length = 0

    def insertElement(self, element):
        # ADD YOUR CODE HERE
        if not self.searchElement(element):
            self.data.append(element)
            self.length += 1
            return True
        return False

    def searchElement(self, element):
        # ADD YOUR CODE HERE
        for i in range(self.length):
            if i == element:
                return True
        return False

Use the cell below to implement the requested API by means of **binary search tree**.

In [8]:
class BinarySearchTreeSet(AbstractSet):
    
    def __init__(self, value=None):
        # ADD YOUR CODE HERE
        self.value = value
        self.left = None
        self.right = None

    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
        if self.value is None:
            self.value = element
        elif element < self.value:
            if self.left is None:
                self.left = BinarySearchTreeSet(element)
                inserted = True
            else:
                self.left.insertElement(element)
        elif element > self.value:
            if self.right is None:
                self.right = BinarySearchTreeSet(element)
            else:
                self.right.insertElement(element)
        return inserted

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE
        if element == self.value:
            found = True
        elif element < self.value and self.left:
            return self.left.searchElement(element)
        elif element > self.value and self.right:
            return self.right.searchElement(element)
        return found

False
True
True
True


Use the cell below to implement the requested API by means of **balanced search tree**.

In [6]:
class Node:
    def __init__(self, value, color=False):
        self.value = value
        self.left = None
        self.right = None
        self.color = color

class BalancedSearchTreeSet(AbstractSet):
    
    def __init__(self):
        # ADD YOUR CODE HERE
        self.root = None

    def insertElement(self, value):
        self.root, inserted = self.insertHelper(self.root, value)
        return inserted

    def insertHelper(self, node, value):
        if node is None:
            return Node(value), True
        if value < node.value:
            node.left, inserted = self.insertHelper(node.left, value)
        elif value > node.value:
            node.right, inserted = self.insertHelper(node.right, value)
        else:
            return node, False
        if self.isRed(node.right) and not self.isRed(node.left):
            node = self.rotateLeft(node)
        if self.isRed(node.left) and self.isRed(node.left.left):
            node = self.rotateRight(node)
        if self.isRed(node.left) and self.isRed(node.right):
            self.flipColor(node)
        return node, inserted

    def searchElement(self, element):
        found = False
        # ADD YOUR CODE HERE
        current = self.root
        while current:
            if current.value == element:
                found = True
                break
            elif current.value > element:
                current = current.left
            else:
                current = current.right
        return found

    def isRed(self, node):
        if node is None:
            return False
        return node.color

    def rotateLeft(self, node):
        x = node.right
        node.right = x.left
        x.left = node
        x.color = node.color
        node.color = True
        return x

    def rotateRight(self, node):
        x = node.left
        node.left = x.right
        x.right = node
        x.color = node.color
        node.color = True
        return x

    def flipColor(self, node):
        node.color = True
        node.left.color = False
        node.right.color = False

Use the cell below to implement the requested API by means of **bloom filter**.

In [6]:
class BloomFilterSet(AbstractSet):
    
    def __init__(self):
        # ADD YOUR CODE HERE

        
        pass           
     
    
        
    def insertElement(self, element):
        inserted = False
        # ADD YOUR CODE HERE
      
        
        return inserted
    
    

    def searchElement(self, element):     
        found = False
        # ADD YOUR CODE HERE

        
        return found    

Use the cell below to implement the **synthetic data generator** as part of your experimental framework.

In [7]:
import string
import random

class TestDataGenerator(AbstractTestDataGenerator):
    
    def __init__(self):
        # ADD YOUR CODE HERE
        
        pass           
        
    def generateData(self, size):     
        # ADD YOUR CODE HERE
        data = [""]*size
        

        return data   



Use the cells below for the python code needed to **fully evaluate your implementations**, first on real data and subsequently on synthetic data (i.e., read data from test files / generate synthetic one, instantiate each of the 4 set implementations in turn, then thorouhgly experiment with insert/search operations and measure their performance).

In [8]:
import timeit

# ADD YOUR TEST CODE HERE TO WORK ON REAL DATA
def read_file(filename):
    my_file = open(filename, 'r')
    lines = my_file.read()
    return lines

def test_spd_SequentialSearchSet():
    testclass = SequentialSearchSet()
    target_values = read_file('test-search.txt')
    for fileName in {'test1-mobydick.txt', 'test2-warpeace.txt', 'test3-dickens.txt'}:
        search_values = read_file(fileName)
        for val in target_values:
            testclass.insertElement(val)
        for val in search_values:
            print(testclass.searchElement(val))
            
def test_spd_BinarySearchTreeSet():
    testclass = BinarySearchTreeSet()
    target_values = read_file('test-search.txt')
    for fileName in {'test1-mobydick.txt', 'test2-warpeace.txt', 'test3-dickens.txt'}:
        search_values = read_file(fileName)
        for val in target_values:
            testclass.insertElement(val)
        for val in search_values:
            print(testclass.searchElement(val))
            
def test_spd_BalancedSearchTreeSet():
    testclass = BalancedSearchTreeSet()
    target_values = read_file('test-search.txt')
    for fileName in {'test1-mobydick.txt', 'test2-warpeace.txt', 'test3-dickens.txt'}:
        search_values = read_file(fileName)
        for val in target_values:
            testclass.insertElement(val)
        for val in search_values:
            print(testclass.searchElement(val))
            
def test_spd_BloomFilterSet():
    testclass = BloomFilterSet()
    target_values = read_file('test-search.txt')
    for fileName in {'test1-mobydick.txt', 'test2-warpeace.txt', 'test3-dickens.txt'}:
        search_values = read_file(fileName)
        for val in target_values:
            testclass.insertElement(val)
        for val in search_values:
            print(testclass.searchElement(val))
            
%timeit test_spd_SequentialSearchSet()
%timeit test_spd_BinarySearchTreeSet()
%timeit test_spd_BalancedSearchTreeSet()
%timeit test_spd_BloodFilterSet()

In [9]:
import timeit

# ADD YOUR TEST CODE HERE TO WORK ON SYNTHETIC DATA



